In [22]:
import json
import numpy as np
import pandas as pd
import networkx as nx

Question 9

In [105]:
# https://geojson.org/
file_geo = "data/los_angeles_censustracts.json"
file_time = "data/los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv"

# Get Node Data
# TODO: node_data reoreganize so src, dst have min value in first index before finding duplicates
with open(file_geo) as f:
    data_geo = json.load(f)
# nodes: locations (lat, long)
node_data = {}  # id: (tract, centroid lat, centroid long)
for row in data_geo["features"]:
    # if row["geometry"]["type"] == "Polygon":
    id = int(row["properties"]["MOVEMENT_ID"])
    tract = int(row["properties"]["TRACT"])
    coords = np.array(row["geometry"]["coordinates"][0])
    # add the centroid coordinates of each polygon to the vertex
    centroid = np.mean(coords, axis=0)
    node_data[id] = (tract, centroid[0], centroid[1])
data_geo = None

# Get Edge Data
# edges: mean travel time between locations
edge_data = pd.read_csv(
    file_time, 
    header=0, 
    usecols=["sourceid", "dstid", "month", "mean_travel_time"]
)
edge_data = edge_data[edge_data["month"] == 12]
mincol = edge_data[["sourceid", "dstid"]].min(axis=1)
maxcol = edge_data[["sourceid", "dstid"]].max(axis=1)
edge_data["sourceid"] = mincol
edge_data["dstid"] = maxcol
print(len(edge_data), "edges before filtering")
print(len(node_data), "nodes before filtering")
edge_data = (
    edge_data.groupby(["sourceid", "dstid"], as_index=False)
    .mean()
    .rename(
        {"sourceid": "source", "dstid": "target", "mean_travel_time": "weight"}, axis=1
    )
)

# Create Graph
G = nx.from_pandas_edgelist(edge_data, "source", "target", "weight")
for id, data in node_data.items():
    if id in G.nodes:
        G.nodes[id]["tract"] = data[0]
        G.nodes[id]["centroid"] = (data[1], data[2])
node_data, edge_data = None, None

# get largest connected component of graph as G
sub = list(max(nx.connected_components(G), key=len))
G = G.subgraph(sub).copy()
print("Number of nodes:", len(G.nodes))
print("Number of edges:", len(G.edges))

C:\Users\Hersh\AppData\Local\Temp\ipykernel_20816\2102091644.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  coords = np.array(row["geometry"]["coordinates"][0])


1692450 edges before filtering
2716 nodes before filtering
Number of nodes: 2649
Number of edges: 1004955


Question 10

In [106]:
# get the mst
mst = nx.minimum_spanning_tree(G)
mst.edges()

EdgeView([(1, 14), (1, 3), (2, 3), (2, 10), (2, 4), (3, 13), (4, 5), (5, 9), (6, 9), (6, 1676), (6, 2015), (7, 8), (8, 9), (10, 11), (12, 18), (12, 22), (15, 1936), (15, 18), (15, 1757), (16, 17), (17, 19), (18, 1758), (18, 19), (18, 1756), (20, 22), (21, 22), (22, 1760), (23, 29), (23, 24), (24, 1972), (25, 2279), (25, 26), (27, 28), (28, 36), (29, 1731), (30, 32), (31, 33), (33, 2258), (33, 1738), (34, 37), (35, 36), (36, 2259), (37, 2019), (52, 51), (52, 53), (58, 311), (58, 76), (58, 59), (60, 76), (60, 2194), (62, 63), (62, 2076), (62, 73), (63, 75), (76, 459), (77, 2061), (77, 81), (77, 1721), (81, 85), (85, 238), (85, 84), (126, 705), (126, 128), (127, 145), (128, 130), (128, 129), (130, 131), (131, 144), (132, 143), (141, 905), (141, 142), (143, 144), (143, 362), (145, 862), (183, 103), (183, 39), (186, 2342), (186, 2060), (187, 2352), (188, 190), (188, 268), (204, 1798), (204, 205), (205, 595), (205, 2392), (238, 2006), (238, 244), (238, 110), (264, 1950), (264, 1776), (265, 2

Question 11

Question 12

Question 13

Question 14

Question 15

Question 16

Question 17

Question 18

Question 19

Question 20

Question 21

Question 22

Question 23

Question 24